tobac example: Tracking of deep convection based on OLR from convection permitting model simulations
==
This example notebook demonstrates the use of tobac to track deep convection based on the outgoing longwave radiation (OLR) from convection permitting simulations.

The simulation results used in this example were performed as part of the ACPC deep convection intercomparison  case study (http://acpcinitiative.org/Docs/ACPC_DCC_Roadmap_171019.pdf) with WRF using the Morrison microphysics scheme. Simulations were performed with a horizontal grid spacing of 4.5 km.

The data used in this example is downloaded from "zenodo link" automatically as part of the notebooks (This only has to be done once for all the tobac example notebooks).

**Import libraries:**

In [9]:
# Import a range of python libraries used in this notebook:
import xarray
import numpy as np
import pandas as pd
import os,sys
import shutil
import datetime
from six.moves import urllib
from glob import glob

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Import tobac itself:
import tobac

In [4]:
# Disable a few warnings:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, append=True)
warnings.filterwarnings('ignore', category=RuntimeWarning, append=True)
warnings.filterwarnings('ignore', category=FutureWarning, append=True)
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)


**Download example data:**  
The actual download is only necessary once for all example notebooks.


In [5]:
data_out='../'

In [7]:
# # Download the data: This only has to be done once for all tobac examples and can take a while
# file_path='https://zenodo.org/record/3195910/files/climate-processes/tobac_example_data-v1.0.1.zip'
# #file_path='http://zenodo..'
# tempfile='temp.zip'
# print('start downloading data')
# request=urllib.request.urlretrieve(file_path,tempfile)
# print('start extracting data')
# shutil.unpack_archive(tempfile,data_out)
# #zf = zipfile.ZipFile(tempfile)
# #zf.extractall(data_out)
# os.remove(tempfile)
# print('data extracted')


start downloading data
start extracting data
data extracted


In [6]:
data_file=os.path.join(data_out,'*','data','Example_input_OLR_model.nc')
data_file = glob(data_file)[0]

In [10]:
#Load Data from downloaded file:
OLR=xarray.open_dataset(data_file)['OLR']

In [11]:
#Set up directory to save output and plots:
savedir='Save'
if not os.path.exists(savedir):
    os.makedirs(savedir)
plot_dir="Plot"
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)

**Feature detection:**  
Feature detection is performed based on OLR field and a set of thresholds.

In [12]:
# Determine temporal and spatial sampling:
dxy,dt=tobac.utils.get_spacings(OLR)

(<xarray.DataArray 'OLR' (time: 96, south_north: 110, west_east: 132)>
[1393920 values with dtype=float32]
Coordinates:
  * time         (time) datetime64[ns] 2013-06-19T19:05:00 ... 2013-06-20T03:00:00
  * south_north  (south_north) int64 156 157 158 159 160 ... 261 262 263 264 265
  * west_east    (west_east) int64 201 202 203 204 205 ... 328 329 330 331 332
    latitude     (south_north, west_east) float32 ...
    longitude    (south_north, west_east) float32 ...
    x            (west_east) float64 ...
    y            (south_north) float64 ...
    x_0          (west_east) int64 ...
    y_0          (south_north) int64 ...
Attributes:
    units:    W m-2,)
{}
converting xarray to iris and back
(<iris 'Cube' of OLR / (W m-2) (time: 96; south_north: 110; west_east: 132)>,)
{}
(4500.0, 300)


In [14]:
# Dictionary containing keyword arguments for feature detection step (Keywords could also be given directly in the function call).
parameters_features={}
parameters_features['position_threshold']='weighted_diff'
parameters_features['sigma_threshold']=0.5
parameters_features['min_num']=4
parameters_features['target']='minimum'
parameters_features['threshold']=[250,225,200,175,150]

In [15]:
# Perform feature detection:
print('starting feature detection')
Features=tobac.themes.tobac_v1.feature_detection_multithreshold(OLR,dxy,**parameters_features)
Features.to_netcdf(os.path.join(savedir,'Features.netcdf'))
print('feature detection performed and saved')


starting feature detection
(      frame  idx     hdim_1      hdim_2  num  threshold_value  feature
0         0    1  48.000000  104.999952    3              250        1
1         0    2  49.000000  100.000000    1              250        2
2         0    4  59.462142   84.000000    2              250        3
3         0    7  65.000000   67.000000    1              250        4
4         0    9  67.000000   62.346732    2              250        5
...     ...  ...        ...         ...  ...              ...      ...
2716     95   28  67.000000  110.000000    1              175     2717
2717     95   29  71.666555  122.473032   62              175     2718
2718     95   30  69.802409   15.037454   10              175     2719
2719     95   31  75.380948  109.634011   19              175     2720
2720     95   34  88.139481  113.625911   68              150     2721

[2721 rows x 7 columns], <iris 'Cube' of OLR / (W m-2) (time: 96; south_north: 110; west_east: 132)>)
{}
      frame  i

**Segmentation:**  
Segmentation is performed with watershedding based on the detected features and a single threshold value.

In [16]:
# Dictionary containing keyword options for the segmentation step:
parameters_segmentation={}
parameters_segmentation['target']='minimum'
parameters_segmentation['method']='watershed'
parameters_segmentation['threshold']=250

In [18]:
# Perform segmentation and save results:
print('Starting segmentation based on OLR.')
Mask_OLR,Features_OLR=tobac.themes.tobac_v1.segmentation(Features,OLR,dxy,**parameters_segmentation)
print('segmentation OLR performed, start saving results to files')
Mask_OLR.to_netcdf(os.path.join(savedir,'Mask_Segmentation_OLR.nc'))                
Features_OLR.to_netcdf(os.path.join(savedir,'Features_OLR.nc'))
print('segmentation OLR performed and saved')

Starting segmentation based on OLR.
<xarray.DataArray 'OLR' (time: 96, south_north: 110, west_east: 132)>
array([[[289.05252, 288.72162, ..., 280.26904, 280.21326],
        [289.42636, 289.08936, ..., 280.36798, 280.3152 ],
        ...,
        [289.93964, 295.0356 , ..., 283.53815, 274.44553],
        [296.27737, 296.89362, ..., 279.30847, 274.90115]],

       [[289.3221 , 288.9656 , ..., 280.25012, 280.19217],
        [289.7054 , 289.34625, ..., 280.35687, 280.3002 ],
        ...,
        [295.6821 , 286.90628, ..., 286.26834, 275.19684],
        [296.15982, 296.80145, ..., 281.44797, 275.65378]],

       ...,

       [[300.36115, 300.2792 , ..., 278.89587, 278.8872 ],
        [300.26105, 300.16174, ..., 278.63638, 278.67062],
        ...,
        [296.05872, 296.319  , ..., 288.2268 , 288.3182 ],
        [295.82617, 296.04742, ..., 287.981  , 287.89703]],

       [[300.3252 , 300.24365, ..., 278.6296 , 278.71307],
        [300.2927 , 300.2258 , ..., 278.47296, 278.5284 ],
        ..

**Trajectory linking:**  
Features are linked into cloud trajectories using the trackpy library (http://soft-matter.github.io/trackpy). This takes the feature positions determined in the feature detection step into account but does not include information on the shape of the identified objects.**

In [19]:
# Arguments for trajectory linking:
parameters_linking={}
parameters_linking['v_max']=20
parameters_linking['stubs']=2
parameters_linking['order']=1
parameters_linking['extrapolate']=1
parameters_linking['memory']=0
parameters_linking['adaptive_stop']=0.2
parameters_linking['adaptive_step']=0.95
parameters_linking['subnetwork_size']=100
parameters_linking['method_linking']= 'predict'

In [20]:
# Perform linking and save results to file:
Track=tobac.themes.tobac_v1.linking_trackpy(Features,OLR,dt=dt,dxy=dxy,**parameters_linking)
Track.to_netcdf(os.path.join(savedir,'Track.nc'))

Frame 95: 18 trajectories present.


**Visualisation:**

In [17]:
# Set extent of maps created in the following cells:
axis_extent=[-95,-89,28,32]        

In [18]:
# Plot map with all individual tracks:
import cartopy.crs as ccrs
fig_map,ax_map=plt.subplots(figsize=(10,10),subplot_kw={'projection': ccrs.PlateCarree()})
ax_map=tobac.plot.map_tracks(Track,axis_extent=axis_extent,axes=ax_map)

In [19]:
# Create animation of tracked clouds and outlines with OLR as a background field
animation_test_tobac=tobac.plot.animation_mask_field(Track,Features,OLR,Mask_OLR,
                                          axis_extent=axis_extent,#figsize=figsize,orientation_colorbar='horizontal',pad_colorbar=0.2,
                                          vmin=80,vmax=330,
                                          plot_outline=True,plot_marker=True,marker_track='x',plot_number=True,plot_features=True)

In [20]:
# Display animation:
from IPython.display import HTML, Image, display
HTML(animation_test_tobac.to_html5_video())

In [ ]:
# # Save animation to file:
# savefile_animation=os.path.join(plot_dir,'Animation.mp4')
# animation_test_tobac.save(savefile_animation,dpi=200)
# print(f'animation saved to {savefile_animation}')

In [21]:
# Lifetimes of tracked clouds:
fig_lifetime,ax_lifetime=plt.subplots()
tobac.plot.plot_lifetime_histogram_bar(Track,axes=ax_lifetime,bin_edges=np.arange(0,200,20),density=False,width_bar=10)
ax_lifetime.set_xlabel('lifetime (min)')
ax_lifetime.set_ylabel('counts')
         

Text(0, 0.5, 'counts')